In [1]:
%autosave 30
%load_ext autoreload
%autoreload 2

Autosaving every 30 seconds


In [2]:
cd ..

/home/huawei123/kwx1991442/code-classification


In [63]:
import logging
from typing import *

from torch import Generator
from torch.utils.data import random_split, DataLoader

from src.baseline.dataloader import UITestsDataset
from src.word2vec.dataloader import W2VClassificationCollator, TrainW2VModel
from src.params import PATH_TEST_UI, PATH_PARSED_CLASSIFUI

logger = logging.getLogger(__name__)

In [64]:
from src.main import parse_args

args = parse_args(True, ["--method", "word2vec", "--tokens-source", "origin"])

In [65]:
args

Namespace(batch_size=4, classifier='bayes', device='cuda', exp_name='TEST', learning_rate=5e-05, liblinear_params='-s 5', max_seq_len=512, method='word2vec', mode='eval', n_epochs=8, save_all=False, save_metrics=False, save_model=False, save_predictions=False, seed=42, tokens_source='origin', traintestsplit=0.7, verbose=False, w2v_concat_method='mean', w2v_epochs=50, w2v_min_count=5, w2v_vector_size=500, w2v_window=500)

In [153]:
if args.mode != 'eval':
    raise NotImplementedError()

# load data
mode = 'train'
tokens_source = args.tokens_source
debug = True  # NOTE!!!!!!!!!!!!!!!!!!!!!!!!!WE ARE IN DEBUG MODE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

if tokens_source == 'origin':
    tests_ui_folder=PATH_TEST_UI
elif tokens_source == 'classifui':
    tests_ui_folder=PATH_PARSED_CLASSIFUI

dataset = UITestsDataset(tests_ui_folder, mode, debug)
print(f"UITestsDataset is initialized in mode {mode} from {tests_ui_folder} "
            f" with {tokens_source=}. {len(dataset)=}")


UITestsDataset is initialized in mode train from data/rust/tests/ui  with tokens_source='origin'. len(dataset)=233


In [68]:
# split data
seed = args.seed
traintestsplit = args.traintestsplit
lengths = [traintestsplit, 1-traintestsplit]

splitgenerator = Generator().manual_seed(seed)
train_set, val_set = random_split(dataset, 
    lengths=lengths, generator=splitgenerator)
print(f"Split generator initialized with {seed=}, {lengths=}.")

Split generator initialized with seed=42, lengths=[0.7, 0.30000000000000004].


In [81]:
concat_method = args.w2v_concat_method #  TODO (NotImplemented)
min_count = args.w2v_min_count
vector_size = args.w2v_vector_size
window = args.w2v_window
epochs = args.w2v_epochs

splitter, w2v_model = TrainW2VModel(train_set, tokens_source, min_count, vector_size, window, epochs).get_model()
collate_fn = W2VClassificationCollator(w2v_model, splitter, dataset.classes)

X_train, Y_train = collate_fn(train_set)
X_val,   Y_val   = collate_fn(val_set)

print(f"{len(dataset.classes)=}, {len(X_train)=}, {len(X_val)=}")

DataLoaders are initialized with batch_size=1
len(dataset.classes)=5, len(X_train)=164, len(X_val)=69


In [155]:
X_train, Y_train = collate_fn(train_set)
X_val,   Y_val   = collate_fn(val_set)

In [121]:
from sklearn.model_selection import train_test_split
X, Y = collate_fn(dataset)
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, train_size=0.7, shuffle=True, random_state=args.seed)

In [102]:
def fix_labels(Y_train, Y_val):
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder().fit(Y_train)
    Y_train = le.fit_transform(Y_train)
    Y_val = le.fit_transform(Y_val)
    return Y_train, Y_val

In [196]:
# Naive Bayes
from sklearn.naive_bayes import GaussianNB, BernoulliNB

# gnb = GaussianNB()
gnb = BernoulliNB()
gnb.fit(X_train, Y_train)
Y_pred = gnb.predict(X_val)
Y_prob = gnb.predict_proba(X_val)

In [305]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

def evaluate(y_val, y_pred, labelencoder) -> Tuple[Dict, Dict, Dict]:
    labels = labelencoder.classes_
    encoded_labels = np.arange(len(labels))

    m_vals, m_arrs, m_count = {}, {}, {}
    m_vals['accuracy'] = accuracy_score(y_val, y_pred)

    m_arrs['precision'], m_arrs['recall'], m_arrs['fbeta_score'], _ = precision_recall_fscore_support(
        y_val, y_pred, average=None, zero_division=1., labels=encoded_labels)

    _, m_count['counts_y_val'] = np.unique(y_val,  return_counts=True)
    m_count['counts_y_pred_x'], m_count['counts_y_pred'] = np.unique(y_pred, return_counts=True)

    m_vals['micro_precision'], m_vals['micro_recall'], m_vals['micro_fbeta_score'], _ = precision_recall_fscore_support(
        y_val, y_pred, average='micro', zero_division=1., labels=encoded_labels)
    m_vals['macro_precision'], m_vals['macro_recall'], m_vals['macro_fbeta_score'], _ = precision_recall_fscore_support(
        y_val, y_pred, average='micro', zero_division=0., labels=encoded_labels)
    return m_vals, m_arrs, m_count

metrics = evaluate(Y_val, Y_pred, collate_fn.labelencoder)
metrics

({'accuracy': 0.9130434782608695,
  'micro_precision': 0.9130434782608695,
  'micro_recall': 0.9130434782608695,
  'micro_fbeta_score': 0.9130434782608695,
  'macro_precision': 0.9130434782608695,
  'macro_recall': 0.9130434782608695,
  'macro_fbeta_score': 0.9130434782608695},
 {'precision': array([1.        , 1.        , 1.        , 1.        , 0.91304348]),
  'recall': array([0., 0., 1., 0., 1.]),
  'fbeta_score': array([0.        , 0.        , 1.        , 0.        , 0.95454545])},
 {'counts_y_val': array([ 1,  4,  1, 63]),
  'counts_y_pred_x': array([4]),
  'counts_y_pred': array([69])})

In [318]:
from src.word2vec.trainer import Trainer

In [323]:
for name in ['logreg', 'forest']: #, 'bayes_bernoulli', 'bayes_gaussian', 'liblinear']:
    tr = Trainer(name, experiment_name='test', labelencoder=collate_fn.labelencoder)
    tr.fit(X_train, Y_train)
    m = tr.eval(X_val, Y_val)

/home/huawei123/kwx1991442/venv/lib/python3.8/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/huawei123/kwx1991442/venv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
10 fits failed out of a total of 20.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/home/huawei123/kwx1991442/venv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/huawei123/kwx1991442/venv/lib/python3.8/site-package